In [1]:
%%capture
!pip install pytorch-pretrained-bert

In [2]:
import math
import torch
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from collections import Counter

In [12]:
with open("../cat_analysis/origin_corpus.txt", "r") as infile:
    errorful = infile.readlines()

In [14]:
with open("../thibo_test/jamspell_usfilter_gector.txt", "r") as infile:
    preds = infile.readlines()

In [18]:
# Load pre-trained model (weights)
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

In [16]:
tokenizer.tokenize(errorful[0])

['keeping</w>',
 'the</w>',
 'secret</w>',
 'of</w>',
 'genetic</w>',
 'testing</w>']

In [17]:
torch.tensor([tokenizer.convert_tokens_to_ids(tokenizer.tokenize(errorful[0]))])

tensor([[ 2621,   481,  2752,   498, 12471,  8448]])

In [19]:
def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss=model(tensor_input, lm_labels=tensor_input)
    return math.exp(loss)



scores = [(score(sents[0]), score(sents[1])) for sents in zip(errorful, preds)] # takes around 7 mins on colab on GPU

RuntimeError: invalid argument 2: dimension -1 out of range of 2D tensor at /pytorch/aten/src/TH/generic/THTensor.cpp:39

In [10]:
Counter([score[0] >= score[1] for score in scores])

Counter()

In [ ]:
mask = [score[0] >= score[1] for score in scores]

# if false: reject prediction

final = [err if m == False else pred for err, pred, m in zip(errorful, jam_gec_pred, mask)]

In [ ]:
with open("jamspell_gector_030_077_filter.txt", "w") as ostr:
    for s in final:
        print(s, file=ostr)